In [ ]:
# pip install -r requirements.txt

In [ ]:
# all imports. use "pip install -r requirements.txt" ^^ if you don't have the needed librarys installed
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from bs4 import BeautifulSoup
from lxml import etree
import re
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
# Start "undetectable" Chrome
options = uc.ChromeOptions()
options.add_argument("--no-first-run --no-service-autorun --password-store=basic")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

# to make shure we don't get detected as a bot
driver = uc.Chrome(options=options, headless=False)

url = "https://www.galaxus.ch/en/s2/producttype/robot-vacuum-cleaners-174?take=204"
driver.get(url)

# Warte auf erste Seite
time.sleep(random.uniform(5, 7))

# scroll down for the lazy loaded items
scroll_height = 0
scroll_step = 2000
new_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # scrolling
    driver.execute_script(f"window.scrollTo({scroll_height}, {scroll_height + scroll_step});")
    time.sleep(random.uniform(0.2, 0.4))
    scroll_height += scroll_step

    # if it gets to the bottom
    if new_height < scroll_height:
        try:
            # choose the button to click
            show_more = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "productListFooter_styled_StyledLoadMoreButton")]'))
            )
            # scroll to the right place to press the button
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", show_more)
            time.sleep(random.uniform(0.4, 0.8))
            show_more.click()
            print("Clicked button")
            # in case the scrolling window gets bigger
            old_height = new_height
            new_height = driver.execute_script("return document.body.scrollHeight")
            time.sleep(random.uniform(1.5, 2.5))

            # savety against a inf loop
            if old_height == new_height:
                # this happens if you are detected as a bot
                print("Button clicked but nothing changed.")
                break
        except:
            # normal out
            print("No more buttons found.")
            break

# save the HTML to search it
soup = BeautifulSoup(driver.page_source, 'lxml')
urls = []

# put all links in a list
for a in soup.find_all("a", href=True):
    href = a["href"]
    # only products
    if href.startswith("/en/s2/product/"):
        full_url = "https://www.galaxus.ch" + href
        if full_url not in urls:
            urls.append(full_url)

# for testing
print(f"\n🔎 Found {len(urls)} product URLs.")
for u in urls[:5]: print(u)

driver.quit()

In [ ]:
data = dict()

In [ ]:
# Start "undetectable" Chrome
options = uc.ChromeOptions()
options.add_argument("--no-first-run --no-service-autorun --password-store=basic")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

# to make shure we don't get detected as a bot
driver = uc.Chrome(options=options, headless=False)

#just for testing
# urls = ["https://www.galaxus.ch/en/s2/product/roborock-s8-maxv-ultra-vacuum-mopping-robot-robot-vacuum-cleaners-43695849", "https://www.galaxus.ch/en/s2/product/dreame-x50-ultra-complete-vacuum-mopping-robot-robot-vacuum-cleaners-53898301"]

for i, url in tqdm(enumerate(urls), total=len(urls)):
    try:
        driver.get(url)
        time.sleep(random.uniform(3, 7))

        soup = BeautifulSoup(driver.page_source, 'lxml')
        dom = etree.HTML(str(soup))

        # here we tryed to do things with the XPATH. Mabe it would have been better with only soup, because there are a lot of trys.
        # but we tought that the base product things are always on the same place (which was wrong)
        # extracting the name

        name_element = dom.xpath('//*[@id="pageContent"]/div/div[1]/div/div/div[2]/div/div[1]/div/h1')
        name = name_element[0].xpath("string()")



        # extracting the price
        try:
            price_element = dom.xpath('//*[@id="pageContent"]/div/div[1]/div/div/div[2]/div/div[1]/span/strong/button/text()')
            price_float = float(re.sub(r"[^\d]", "", price_element[0]))
        except:
            # can happen if the product is returned
            price_element = dom.xpath('//*[@id="pageContent"]/div/div[1]/div[1]/div/div[2]/div/div[1]/span/strong/text()')
            price_float = float(re.sub(r"[^\d]", "", price_element[0]))
        
        
        if data.get(str(name)) is None:
            data[str(name)] = dict()
        else:
            # if theoretical only the color changes, we just overwrite the old datapoint. But they would be simmilar anyway
            # rename the old one
            data[name + "_" + str(data.get(name).get("price"))] = data.get(name)
            data.pop(name)
            # create the new one
            name = name + "_" + str(price_float)
            data[name] = dict()
            
        data.get(name).update({"price" : price_float})

        # extracting the rating
        try:
            rating_element = dom.xpath('//*[@id="pageContent"]/div/div[1]/div/div/div[2]/div/div[2]/div[1]/a/span[3]')                   
            rating_float = float(rating_element[0].get("aria-label").split()[0])
        except:
            try:
                # only a small change, I thing it happens when there is an additional price reduction window
                rating_element = dom.xpath('//*[@id="pageContent"]/div/div[1]/div/div/div[2]/div/div[3]/div[1]/a/span[3]')
                rating_float = float(rating_element[0].get("aria-label").split()[0])
            except:
                # happens if there is no rating
                rating_float = None
        data.get(name).update({"rating" : rating_float})

        # extracting the rating
        try:
            rating_count_element = dom.xpath('//*[@id="pageContent"]/div/div[1]/div/div/div[2]/div/div[2]/div[1]/a/span[1]')
            rating_count_int = int(rating_count_element[0].xpath("string()"))
        except:
            try:
                # only a small change, I thing it happens when there is an additional price reduction window
                rating_count_element = dom.xpath('//*[@id="pageContent"]/div/div[1]/div/div/div[2]/div/div[3]/div[1]/a/span[1]')
                rating_count_int = int(rating_count_element[0].xpath("string()"))
            except:
                # happens if there is no rating
                rating_float = None
        data.get(name).update({"rating_count" : rating_count_int})

        # here we did it only with soup
        # save everything in the tables
        for table in soup.find_all(class_="specificationTable_styled_SpecificationTableStyled__G_W91"):
            table_name = table.find("caption").get_text(strip=True) if table.find("caption") else "Unknown"
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                if len(cells) == 2:
                    key = cells[0].get_text(separator=" ", strip=True)
                    values = cells[1].find_all("span", class_="value_StyledValue__KCmTz")
                    value_texts = [v.get_text(" ", strip=True) for v in values]
                    value = ", ".join(value_texts).replace("\xa0", " ")
                    data.get(name).update({table_name + "  " + key : value})
        driver.delete_all_cookies()
    except:
        # if something goes wrong / or the site couldn't be loadet out of some reason
        # not the nicest way, but the last run didn't print any problems anyway
        print(f"There was a Problem with: {i} {url}")
        pass

driver.quit()

In [ ]:
# example of the dict
print(list(data.keys())[0])
data[list(data.keys())[0]]

In [ ]:
# change the dict to DataFrame and put the name in a new line
df = pd.DataFrame.from_dict(data, orient='index')
df = df.reset_index().rename(columns={"index": "product_name"})
print(df.shape)
# save as csv
df.to_csv("robot_vacuums.csv", index=False)
df